# Synthetic ice stream

In this demo, we'll simulate the evolution of an ice stream with both grounded and floating parts.
This scenario is a good bit more complex than the floating ice shelf we simulated in the previous demo.
First, there are more fields.
For floating ice shelves, we only had to consider the velocity, thickness, and fluidity.
For grounded glaciers, we also have to include the surface elevation and bed friction.
The diagnostic equation for an ice stream is similar to that of a floating ice shelf, but for the addition of a term for basal friction:

$$\nabla\cdot hM - C|u|^{1/m - 1}u - \rho gh\nabla s = 0,$$

where $m$ is the *sliding exponent* and $C$ is the *sliding coefficient*.
Second, the position of the grounding line -- the contour across which the glacier is thin enough to go afloat -- can migrate in time.
Accurately predicting grounding line migration is a [major problem](https://doi.org/10.1029/2004JF000202) in glacier flow modeling.

As a test case, we'll use a geometry and input data from the third phase of the *Marine Ice Sheet Model Intercomparison Project*, or MISMIP+.
If you're not already familiar with the term, model intercomparison projects are standardized computational experiments that researchers design in order to test the degree to which different software packages produce outputs that do or don't agree with each other.
There have been several ice sheet model intercomparison projects in the past, but they're common in other disciplines as well.
For example, there have also been atmospheric model and paleoclimate model intercomparison projects.
MISMIP+ has become a standard test case for evaluating glacier flow models and we'll revisit it again several times in the guides that follow.
The experimental setup for MISMIP+ is described in [Asay-Davis et al. (2016)](https://doi.org/10.5194/gmd-9-2471-2016).

### Geometry and input data

The experiment uses an elongated geometry roughly the size of a glacier catchment -- 640 km from the ice divide to the terminus and 80 km across.

In [ ]:
import firedrake

Lx, Ly = 640e3, 80e3
ny = 16
nx = int(Lx / Ly) * ny
area = Lx * Ly

mesh = firedrake.RectangleMesh(nx, ny, Lx, Ly)
Q = firedrake.FunctionSpace(mesh, family="CG", degree=2)
V = firedrake.VectorFunctionSpace(mesh, family="CG", degree=2)

The MISMIP+ experiment defines a profile for the ice bed that is a 6th-order polynomial in $x$ and an exponential in $y$.
The bed shape was designed to create a bench or bedrock high in the $x$ direction that the ice could ground on and, given enough of a meltwater kick, to retreat off of.
The shape in the $y$ direction is designed to funnel ice off of the side walls and create geometric constrictions.

In [ ]:
from firedrake import exp, max_value, Constant, as_vector, interpolate, dx

x, y = firedrake.SpatialCoordinate(mesh)
x_c = Constant(300e3)
X = x / x_c

B_0 = Constant(-150)
B_2 = Constant(-728.8)
B_4 = Constant(343.91)
B_6 = Constant(-50.57)
B_x = B_0 + B_2 * X ** 2 + B_4 * X ** 4 + B_6 * X ** 6

f_c = Constant(4e3)
d_c = Constant(500)
w_c = Constant(24e3)

B_y = d_c * (
    1 / (1 + exp(-2 * (y - Ly / 2 - w_c) / f_c)) +
    1 / (1 + exp(+2 * (y - Ly / 2 + w_c) / f_c))
)

z_deep = Constant(-720)
z_b = interpolate(max_value(B_x + B_y, z_deep), Q)

In case all of that algebra wasn't totally crystal clear, here's a picture.

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

fig = plt.figure()
axes = fig.add_subplot(projection="3d")
firedrake.trisurf(z_b, axes=axes);

The experiment further sets the default values for the ice fluidity and friction coefficients.

In [ ]:
A = Constant(20)
C = Constant(1e-2)

In the previous examples, we've initialized a model object and then used it directly to compute velocities and thicknesses.
These model objects have sensible defaults for how the physics are parameterized.
For example, we pass in the ice fluidity coefficient $A$, which has units of strain rate $\times$ stress${}^{-n}$, where $n = 3$ is the exponent in Glen's flow law.
That's all fine for getting started, but what if you don't like our defaults?
What if you instead wanted to use, say, the rheology coefficient $B = A^{-1/n}$ instead?
Or what if you wanted to change the friction law completely?

This brings us to one of the most important design features of icepack: **you can customize the physics parameterizations.**
To do so, all you have to do is write a Python function to compute the relevant part of the action functional and pass that function when you create the model.
Very often, you can reuse some of the functions in icepack with a little bit of modification rather than write a completely new one.

The default in the `IceStream` class is to take in a friction coefficient $C$.
We want to change this default behavior to also include a *ramping* factor that smoothly dials the friction down towards the grounding line as a function of the ice thickness and surface elevation.
Without this smoothing effect, the system can experience a sudden shock across the grounding line, which manifests as wiggly numerical artifacts.
The following function works as a wrapper around the default parameterization, multiplying the friction coefficient by the ramping factor.

In [ ]:
from icepack.constants import (
    ice_density as ρ_I,
    water_density as ρ_W,
    gravity as g,
)
import icepack.models.friction


def weertman_friction_with_ramp(**kwargs):
    u = kwargs["velocity"]
    h = kwargs["thickness"]
    s = kwargs["surface"]
    C = kwargs["friction"]

    p_W = ρ_W * g * firedrake.max_value(0, -(s - h))
    p_I = ρ_I * g * h
    ϕ = 1 - p_W / p_I
    return icepack.models.friction.bed_friction(velocity=u, friction=C * ϕ)

Now we can create a model object that uses our new parameterization by passing it to the constructor for the `IceStream` class.
We're calling the model object `model_weertman` because, at the end of this demo, we'll compare with something quite different -- the Schoof model for basal sliding.
The solver object is created in the same way as before.
Customizing the physics changes what problem we're solving, but the algorithms that we use to approximate the solution are identical.

In [ ]:
model_weertman = icepack.models.IceStream(friction=weertman_friction_with_ramp)
opts = {"dirichlet_ids": [1], "side_wall_ids": [3, 4], "tolerance": 1e-6}
solver_weertman = icepack.solvers.FlowSolver(model_weertman, **opts)

### Modeling

Now the good part -- taking our initial glacier state and projecting it forward until it reaches a steady state.
The MISMIP+ experiment specifies an accumulation rate of 30 cm / year for spinning up the ice stream.

In [ ]:
a = Constant(0.3)

We don't have a great way to know ahead of time what the steady state of the simulation will be.
Instead, we'll start with a very blunt initial guess: a constant ice thickness of 100m everywhere.

In [ ]:
h_0 = interpolate(Constant(100), Q)
s_0 = icepack.compute_surface(thickness=h_0, bed=z_b)

x = firedrake.SpatialCoordinate(mesh)[0]
u_0 = solver_weertman.diagnostic_solve(
    velocity=interpolate(as_vector((90 * x / Lx, 0)), V),
    thickness=h_0,
    surface=s_0,
    fluidity=A,
    friction=C,
)

We'll then spin this up to steady state for long enough to propagate out any initial transients.
The MISMIP+ paper gives a value of a few thousand years as sufficient to get extremely close to steady state.
If you didn't know ahead of time, you could also find this by trial and error.

In [ ]:
final_time = 3600.0
dt = 3.0
num_steps = int(final_time / dt)

h = h_0.copy(deepcopy=True)
u = u_0.copy(deepcopy=True)

In [ ]:
import tqdm

for step in tqdm.trange(num_steps):
    h = solver_weertman.prognostic_solve(
        dt,
        thickness=h,
        velocity=u,
        accumulation=a,
        thickness_inflow=h_0,
    )
    s = icepack.compute_surface(thickness=h, bed=z_b)

    u = solver_weertman.diagnostic_solve(
        velocity=u,
        thickness=h,
        surface=s,
        fluidity=A,
        friction=C,
    )

Notice that we have one extra step in the middle of the simulation loop.
For ice shelves, all we had to do was (1) update the thickness according to the prognostic equation and then (2) compute the new ice velocity corresponding to this thickness.
Since ice shelves are floating, we can calculate the surface elevation as a function of the ice thickness knowing the densities of ice and ocean water, and this is baked into the formulation of the problem.
Ice streams, on the other hand, can be grounded or floating.
After we update the ice thickness, we need to explicitly calculate a new surface elevation in a way that accounts for whether the ice is floating or grounded, and that's what the function `icepack.compute_surface` does.
Let's see what the results look like.

In [ ]:
import icepack.plot

fig, axes = icepack.plot.subplots()
colors = icepack.plot.tripcolor(h, axes=axes)
fig.colorbar(colors, label="meters", fraction=0.012, pad=0.04);

In [ ]:
fig, axes = icepack.plot.subplots()
colors = icepack.plot.tripcolor(u, axes=axes)
fig.colorbar(colors, label="meters/year", fraction=0.012, pad=0.04);

We can get an idea for how close or far away the system is from equilibrium by plotting the total mass imbalance.

In [ ]:
from firedrake import div

f = firedrake.interpolate(a - div(h * u), Q)
fig, axes = icepack.plot.subplots()
colors = icepack.plot.tripcolor(f, vmin=-0.5, vmax=+0.5, cmap="RdBu", axes=axes)
fig.colorbar(colors, label="meters/year", fraction=0.012, pad=0.04);

Finally, we can draw some transects through the domain to see what the surface, bed, and ice base elevation look like in the $x, z$-plane.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

xs = np.array([(Lx * k / nx, 0) for k in range(nx + 1)])

ss = np.array(s.at(xs, tolerance=1e-10))
hs = np.array(h.at(xs, tolerance=1e-10))
bs = np.array(z_b.at(xs, tolerance=1e-10))

fig, axes = plt.subplots()
axes.plot(xs[:, 0] / 1e3, bs, color="black")
axes.plot(xs[:, 0] / 1e3, ss - hs, color="blue")
axes.plot(xs[:, 0] / 1e3, ss, color="blue")
axes.set_xlabel("distance along centerline (km)")
axes.set_ylabel("elevation (m)");

There are a few things to observe about this.
First, the glacier exhibits the typical drop in thickness gradient as it crosses the grounding line, and there's a bit of a hump or bulge right upstream of it.
This matches pretty nicely with what we see in real observations.

Second, there are some oscillatory wiggles at the ice divide on the left side of the domain.
This does not match nicely with what we see in real observations at all.
These wiggles are purely a numerical artifact of the system adjusting on spatial scales smaller than what our mesh can resolve.
We can fix this by using a mesh with higher resolution near the ice divide; we'll cover adaptive mesh refinement in a later tutorial.
These sorts of problems are very common and you should know what they look like.

### Recap

The demonstration above shows how to simulate the evolution of a grounded ice stream; while there are more fields to keep track of than for a floating ice shelf, the basic principles are still the same.

Coming up with physically plausible synthetic test cases is difficult.
The variables that we can control are the bed geometry, the accumulation rate, and the ice flux at inflow.
To a certain extent, we can pick the friction coefficient, but this can change in ways that are difficult to predict as parts of the glacier begin to float.
Our initial state was far out of mass balance and we let it evolve towards equilibrium, but we don't know ahead of time what surface slope or stretching rate the glacier will settle on in the end.

The eventual steady state could be physically implausible for reasons that weren't obvious from the start.
For example, if the ice strain rate increases beyond what we expect after a few years of model time, the mass flux at the inflow boundary could be much larger than what the influx and accumulation rate can supply, leading to an unphysically sharp drop in the surface.
Preparing demonstrations like this one can require lots of trial and error to get the parameters just right, and that's to be expected.

The bed elevation we used sloped down towards the ocean; had we instead used a bed elevation that sloped down going inland, the configuration would be unstable due to marine ice sheet instability.
An interesting variation on this demo could be to use a bed geometry with several stable pinning points, interspersed with unstable retrograde slopes.

### Changing the model physics

In the first part of this demo, we used the usual Weertman sliding law, where the basal shear stress is proportional to some power of the ice sliding speed.
This sliding law assumes that the glaciers slide over hard beds by the process of [regelation](https://en.wikipedia.org/wiki/Regelation).
Where the glacier flows over deformable sediments, the sliding relation could instead approach plasticity -- the stress depends only on the direction and not the magnitude of the velocity.
Other sliding laws have been proposed that exhibit this feature, for example the law proposed in Schoof (2005), The effect of cavitation on glacier sliding.

In icepack, all of the diagnostic model physics is prescribed through an action principle.
Rather than specify how the shear stress depends on the ice velocity, we specify the frictional energy dissipation.
The stress is then the functional derivative of the dissipation with respect to velocity.
For the Weertman sliding law

$$\tau_b = -C|u|^{\frac{1}{m} - 1}u,$$

the energy dissipation is

$$E(u) = \int \frac{m}{m + 1}C|u|^{\frac{1}{m} + 1}dx.$$

In the first part of the demo, we showed how to slightly modify the physics by adding a thickness- and elevation-dependent factor to reduce the friction when the glacier bed is below sea level.
In the following, we'll show how to use a completely different model of glacier sliding.

### The Schoof sliding law

To modify the Weertman sliding law for high ice speeds, we need for the energy dissipation density to be asymptotically like $|u|^1$ for large $u$, but like $|u|^{1/m + 1}$ for small $u$.
"Small" and "large" will be defined relative to some threshold or critical speed $u_c$.
While it doesn't exactly reproduce the Schoof sliding law itself, the following functional gives the right asymptotic behavior:

$$E(u) = \int \tau_c\left(\left(u_c^{\frac{1}{m} + 1} + |u|^{\frac{1}{m} + 1}\right)^{\frac{m}{m + 1}} - u_c\right) dx$$

The extra factor of $-u_c$ is there to make it so that $E(0) = 0$, i.e. there is no dissipation when there is no flow.

If we're going to use this more complex friction law, we also have to decide what values to use for the critical stress $\tau_c$ and velocity $u_c$.
The MISMIP+ experiment suggests taking the critical stress to be equal to half the pressure exerted by the ice above flotation.
Just like we did before with the Weertman sliding law, this choice will give us the right ramping factor as we get towards the grounding line.
We'll then define the critical speed $u_c$ in terms of the sliding coefficient $C$ and the critical stress:

$$u_c = (\tau_c / C)^m.$$

Since $\tau_c \to 0$ near the grounding line, this means that the sliding law will be closer to plasticity there as well.

The following function returns the kernel of the action functional associated to this sliding law, given the ice velocity, elevation, and the yield stress.
Note that we only need to provide the inside of the integral -- the model object applies the measures (`dx`, `ds`, etc.) for you.

In [ ]:
def schoof_friction(**kwargs):
    u = kwargs["velocity"]
    h = kwargs["thickness"]
    s = kwargs["surface"]
    C = kwargs["friction"]

    p_W = ρ_W * g * max_value(0, -(s - h))
    p_I = ρ_I * g * h
    N = max_value(0, p_I - p_W)
    τ_c = N / 2

    u_c = (τ_c / C) ** m
    u_b = sqrt(inner(u, u))

    return τ_c * ((u_c ** (1 / m + 1) + u_b ** (1 / m + 1)) ** (m / (m + 1)) - u_c)

Before we go on, let's take a look at what the basal shear stress actually looks like.
Recall that the quantity we defined above is the rate of free energy dissipation due to frictional stress.
To get the actual basal shear, we'll take the derivative of this quantity with respect to the velocity $u$.
The analytical formula for the stress is

$$\tau = -\frac{\tau_c|u|^{\frac{1}{m} - 1}u}{\left(u_c^{\frac{1}{m} + 1} + |u|^{\frac{1}{m} + 1}\right)^{\frac{1}{m + 1}}}.$$

Observe how the magnitude of the basal shear is asymptotic to $|u|^{1/m}$ when $|u| \ll u_c$, but to $\tau_c$ when $|u| \gg u_c$.

To calculate the basal shear, we could painstakingly write out this whole formula in Python.
Or we could take advantage of the symbolic differentiation features of the form language by first creating the free energy dissipation `F` and then calling `firedrake.diff` on it.

In [ ]:
F = schoof_friction(
    velocity=u,
    thickness=h,
    surface=s,
    friction=C,
)
τ = firedrake.interpolate(-firedrake.diff(F, u), V)

In [ ]:
fig, axes = icepack.plot.subplots()
colors = firedrake.tripcolor(τ, vmin=0, vmax=0.15, axes=axes)
fig.colorbar(colors, label='MPa', fraction=0.012, pad=0.04);

This matches up with what we expect -- higher stress at the grounded part of the ice stream, which then decreases to 0 towards the grounding line.

Now we can create a new `IceStream` object that will use this function to calculate the action rather than the usual Weertman sliding law:

In [ ]:
model_schoof = icepack.models.IceStream(friction=schoof_friction)

The Schoof friction functional that we defined above takes in more parameters than the usual Weertman law, but thanks to keyword arguments in Python, everything gets passed to the right place despite this change.
Let's do a diagnostic solve with this new model and see how different it is from the velocity obtained with the Weertman sliding law.

In [ ]:
solver_schoof = icepack.solvers.FlowSolver(model_schoof, **opts)
u_schoof = solver_schoof.diagnostic_solve(
    velocity=u,
    thickness=h,
    surface=s,
    fluidity=A,
    friction=C,
)

In [ ]:
fig, axes = icepack.plot.subplots()
colors = icepack.plot.tripcolor(u_schoof, axes=axes)
fig.colorbar(colors, label='meters/year', fraction=0.012, pad=0.04);

In [ ]:
icepack.norm(u - u_schoof)

Just as we expect -- the resulting velocities are practically the same.
We picked the parameters in the Schoof sliding law to give the same basal shear stress as with the Weertman law, so we should get the same velocity.

### Response to perturbations

The steady state we've found is roughly an equilibrium for both the Weertman and Schoof sliding laws.
To conclude this demo, we'll increase the melt rate under the floating ice tongue and see how the system responds.
To perturb the system out of its current equilibrium state, we'll add an extra 1 m/year of melting in the right-hand side of the domain.
If we wanted to be more physically realistic about things, we might set the melt rate to be a function of depth below sea level, or even use a plume model.

In [ ]:
from firedrake import conditional
a = firedrake.interpolate(
    a_in + δa * x / Lx - conditional(x/Lx > 0.5, 1.0, 0.0), Q
)

We'll wrap up the same loop as we had before in a function that takes in the model (Weertman or Schoof) as an argument.
This saves a bit of repetition and makes it easier to parallelize later.

In [ ]:
num_years = 100
timesteps_per_year = 2

δt = 1.0 / timesteps_per_year
num_timesteps = num_years * timesteps_per_year

def run_simulation(solver, h, s, u, **kwargs):
    for step in tqdm.trange(num_timesteps):
        h = solver.prognostic_solve(
            δt,
            thickness=h,
            velocity=u,
            accumulation=a,
            thickness_inflow=h0,
        )
        s = icepack.compute_surface(thickness=h, bed=b)
        
        u = solver.diagnostic_solve(
            velocity=u,
            thickness=h,
            surface=s,
            **kwargs,
        )

    return h, s, u

h_weertman, s_weertman, u_weertman = run_simulation(
    solver_weertman, h, s, u, fluidity=A, friction=C
)

h_schoof, s_schoof, u_schoof = run_simulation(
    solver_schoof, h, s, u_schoof, fluidity=A, yield_stress=τ_0
)

Finally, we'll plot a transect of the difference in thickness between the Weertman and Schoof laws.
The glacier loses more mass over the grounded region and less under the ice shelf with the Schoof law than with the Weertman law.
When the sliding relation becomes more plastic, the bed can take up less of the increased driving stress resulting from steeper surface slopes, so that changes under the ice shelf are evident further upstream.

In [ ]:
hs_weertman = np.array(h_weertman.at(xs, tolerance=1e-10))
hs_schoof = np.array(h_schoof.at(xs, tolerance=1e-10))

fig, axes = plt.subplots()
axes.plot(xs[:, 0] / 1e3, hs_weertman - hs_schoof, color="black")
axes.set_xlabel("distance along centerline (km)")
axes.set_ylabel("thickness difference (m)");

### Conclusion

The Schoof sliding law is, arguably, more physically realistic than the Weertman sliding law for fast-flowing outlet glaciers or ice streams.
While icepack defaults to using the Weertman sliding law, replacing it with the Schoof law consists of only a few lines of code.
First, we have to write a function that calculates the energy dissipation from Schoof-type sliding.
We then have to pass this function to the constructor of the `IceStream` model object we use to calculate the ice velocity.
By passing in all the input fields to the `diagnostic_solve` method as keyword arguments, we don't have to rewrite or override any methods of `IceStream`, despite the fact that there are more input fields to the Schoof sliding law than there are for the Weertman law.

In switching to the Schoof sliding law, we showed how the physics could be modified to take in a possibly arbitrary number of different fields beyond the default physics parameterization.
These additional fields could themselves be computed by solving other PDEs.
For example, we could define the basal shear stress to be a function of the effective water pressure, which would then solve for using some model of subglacial hydrology.